In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df = pd.read_excel('./documents/aug_nine_var.xlsx').iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21::].to_numpy()

In [4]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X.shape

(395, 21)

In [5]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [6]:
def my_loss(real_data, gen_data):
    dist = tf.math.sqrt(tf.reduce_sum(tf.math.square(tf.math.subtract(real_data, gen_data))))
    y_pred = dnn_model(gen_data)

    a = tf.math.subtract(y_pred, 42)
    b = dist

    loss = tf.math.add(a, b)
    return loss

In [7]:
generator = tf.keras.Sequential()
generator.add(tf.keras.layers.Dense(128, input_dim=21, activation='relu'))
generator.add(tf.keras.layers.Dense(64, activation='relu'))
generator.add(tf.keras.layers.Dense(32, activation='relu'))
generator.add(tf.keras.layers.Dense(16, activation='relu'))
generator.add(tf.keras.layers.Dense(8, activation='relu'))
generator.add(tf.keras.layers.Dense(4, activation='relu'))
generator.add(tf.keras.layers.Dense(21, activation='sigmoid'))

generator.compile(loss=my_loss, optimizer='adam')

In [8]:
def return_latest():
    df = pd.read_excel('./documents/aug_nine_var.xlsx').iloc[:,1::]
    X = df.iloc[:,0:21]
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    latest = X[-1].reshape(1, 21)

    return tf.cast(latest, dtype=tf.float32)

In [9]:
result = []
pop_result = []
for i in range(1000):
    noise = tf.random.normal([1, 21])
    X_pred = generator.predict(noise, verbose=0)
    real_data = return_latest()
    loss = my_loss(real_data, X_pred)
    generator.train_on_batch(X_pred, real_data)

    pop = dnn_model.predict(X_pred, verbose=0)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[0.7149137]], pop: [[40.92142]]
epoch: 1, loss: [[0.7064395]], pop: [[40.913696]]
epoch: 2, loss: [[0.6979579]], pop: [[40.905964]]
epoch: 3, loss: [[0.6894845]], pop: [[40.89824]]
epoch: 4, loss: [[0.7011266]], pop: [[40.911663]]
epoch: 5, loss: [[0.67253494]], pop: [[40.882786]]
epoch: 6, loss: [[0.6640513]], pop: [[40.87505]]
epoch: 7, loss: [[0.65557945]], pop: [[40.867325]]
epoch: 8, loss: [[0.6471044]], pop: [[40.859596]]
epoch: 9, loss: [[0.6386297]], pop: [[40.851868]]
epoch: 10, loss: [[0.63015914]], pop: [[40.844143]]
epoch: 11, loss: [[0.62168527]], pop: [[40.836414]]
epoch: 12, loss: [[0.61321175]], pop: [[40.828686]]
epoch: 13, loss: [[0.60474265]], pop: [[40.82096]]
epoch: 14, loss: [[0.5962738]], pop: [[40.813236]]
epoch: 15, loss: [[0.5877979]], pop: [[40.805504]]
epoch: 16, loss: [[0.5793338]], pop: [[40.797783]]
epoch: 17, loss: [[0.5708587]], pop: [[40.79005]]
epoch: 18, loss: [[0.56239176]], pop: [[40.782326]]
epoch: 19, loss: [[0.5539212]], pop: [[

In [10]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [11]:
df = np.append(X_result, pop_result, axis=1)

In [12]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/basic_formula.xlsx')